In [86]:
import pandas as pd
import numpy as np
from Reducción_Memoria import reduce_mem_usage
from Control_de_Daños import control_de_danos
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import json

import joblib

In [87]:
pd.set_option('display.max_columns', None)

COLS_TARGET_ENC = ['trip_origin_city', 'trip_destination_city', 'pos', 'rt_cc', 'trip_rt'] # , 'pos', 'rt_group'
COLS_ONE_HOT = ['trip_type', 'rt_group']

SEED = 42
TEST_SIZE_GENERAL = 0.1
TEST_SIZE_PAIS = 0.1
DATA_DIR = 'Data/'

In [88]:
CITY_TO_COUNTRY = {
    # --- CHILE (CL) ---
    'SCL': 'CL', 'ANF': 'CL', 'ARI': 'CL', 'BBA': 'CL', 'CCP': 'CL',
    'CJC': 'CL', 'IQQ': 'CL', 'LSC': 'CL', 'PMC': 'CL', 'ZCO': 'CL',

    # --- ARGENTINA (AR) ---
    'EZE': 'AR', 'AEP': 'AR', 'BUE': 'AR',
    'BRC': 'AR', 'COR': 'AR', 'CPC': 'AR', 'CRD': 'AR', 'FTE': 'AR',
    'IGR': 'AR', 'MDZ': 'AR', 'NQN': 'AR', 'REL': 'AR', 'RES': 'AR',
    'SLA': 'AR', 'TUC': 'AR', 'USH': 'AR',

    # --- COLOMBIA (CO) ---
    'BOG': 'CO', 'MDE': 'CO', 'CTG': 'CO', 'CLO': 'CO', 'ADZ': 'CO',
    'BAQ': 'CO', 'CUC': 'CO', 'MTR': 'CO', 'PEI': 'CO', 'SMR': 'CO',

    # --- PERÚ (PE) ---
    'LIM': 'PE', 'AQP': 'PE', 'CUZ': 'PE', 'PIU': 'PE', 'TRU': 'PE',
    'CIX': 'PE', 'CJA': 'PE', 'TPP': 'PE',

    # --- BRASIL (BR) ---
    'FLN': 'BR', 'IGU': 'BR', 'REC': 'BR', 'RIO': 'BR', 'SAO': 'BR',

    # --- OTROS ---
    'ASU': 'PY', # Paraguay
    'MVD': 'UY', # Uruguay
    'UIO': 'EC', # Ecuador
    # Agrego US por seguridad (aunque no salgan en tu lista de origen, el POS existe)
    'MIA': 'US', 'FLL': 'US', 'JFK': 'US'
}

In [89]:
def exportar_target_encoding_a_json(X_train, y_train, columnas_a_codificar, ruta_salida):
    """
    Genera un diccionario con los mappings de Target Encoding por columna y lo guarda en JSON.
    Estructura resultante:
    {
      "global_mean": 0.1234,
      "columns": {
        "col1": { "mapping": { "catA": 0.12, "catB": null }, "else": 0.1234 },
        ...
      }
    }
    """
    temp_df = X_train.copy()
    temp_df['target'] = y_train
    global_mean = float(y_train.mean())

    result = {'global_mean': global_mean, 'columns': {}}

    for col in columnas_a_codificar:
        mapping = temp_df.groupby(col, observed=False)['target'].mean()
        col_map = {}
        for categoria, valor in mapping.items():
            # Normalizar clave para JSON (usar 'NULL' para NaNs)
            key = 'NULL' if pd.isna(categoria) else str(categoria)
            # Convertir valor a float o None si es NaN
            val = None if pd.isna(valor) else float(valor)
            col_map[key] = val

        result['columns'][col] = {'mapping': col_map, 'else': global_mean}

    # Guardar JSON
    with open(ruta_salida, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)

    return result

In [90]:
def limpiar_moneda(serie):
    """
    Convierte strings de moneda con formato '1.000,00' a float.
    Si ya es numérico, lo devuelve tal cual (evita el error de los notebooks 03/04).
    """
    if pd.api.types.is_numeric_dtype(serie):
        return serie

    # Convierte a string, quita puntos de mil, cambia coma por punto
    clean = serie.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return pd.to_numeric(clean, errors='coerce').fillna(0)


In [91]:
def preparacion_maestra(df_input):
    df = df_input.copy()

    # 1. Imputación de Nulos Lógicos
    cols_zero = ['length_of_stay', 'has_weekends', 'working_days']
    for col in cols_zero:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # 2. Limpieza de Monedas
    cols_dinero = ['ancillary_revenue', 'fare_revenue']
    for col in cols_dinero:
        if col in df.columns:
            df[col] = limpiar_moneda(df[col])

    # 3. Creación del TARGET (Regla de Negocio)
    if 'motivo_de_viaje' in df.columns:
        mapa = {'Trabajo': 1, 'Vacaciones': 0, 'Familiar': 0}
        df['target'] = df['motivo_de_viaje'].map(mapa)

    # 4. Creamos columna que relaciona 'pos' con 'rt_group', si compra en chile para volar fuera es ocio
    def obtener_pais_vuelo(row):
        grupo = str(row['rt_group'])

        # Caso 1: Vuelo Doméstico (DOMCO, DOMPE, DOMAR...)
        if grupo.startswith('DOM'):
            return grupo[-2:] # Retorna CO, PE, AR, CL

        # Caso 2: Vuelo Internacional (INTER)
        # Miramos el origen. Si sale de LIM es vuelo "Peruano" en origen.
        orig = row['trip_origin_city']
        return CITY_TO_COUNTRY.get(orig, 'OTRO')


    if all(c in df.columns for c in ['has_bag', 'has_pb', 'has_seat']):
        df['perfil_ejecutivo_ligero'] = (
            (df['has_bag'] == 0) &
            ((df['has_pb'] == 1) | (df['has_seat'] == 1))
        ).astype(int)
    else:
        df['perfil_ejecutivo_ligero'] = 0

    # Solo aplica si NO es Solo Ida (trip_type=2)
    df['es_viaje_corto'] = (
        (df['trip_type'] == 2) &          # Condición 1: Que sea Ida y Vuelta
        (df['length_of_stay'] <= 2) &     # Condición 2: Max 2 días
        (df['length_of_stay'] > 0)        # Condición 3: Que no sea 0 (por seguridad)
    ).astype(int)

    # 7. Ingeniería de Fechas (Cíclicas)
    if 'trip_start_date' in df.columns:
        fechas = pd.to_datetime(df['trip_start_date'], dayfirst=True, errors='coerce')

        # Ciclo Mensual (1-12)
        df['month_sin'] = np.sin(2 * np.pi * fechas.dt.month / 12)
        df['month_cos'] = np.cos(2 * np.pi * fechas.dt.month / 12)

        # Ciclo Semanal (0-6)
        df['dow_sin'] = np.sin(2 * np.pi * fechas.dt.dayofweek / 7)
        df['dow_cos'] = np.cos(2 * np.pi * fechas.dt.dayofweek / 7)

        df.drop(columns=['trip_start_date'], inplace=True)

    # 5. Eliminación de Columnas Ruidosas o Redundantes
    cols_drop = ['has_promo_class', 'promocode', 'recordlocator','discounted_fare_revenue_dc', 'discounted_fare_revenue_pc', 'discount_perc', 'infants', 'children', 'status', 'booking_hour', 'booking_dow', 'booking_weeknumber', 'booking_weeknumber', 'bookingid', 'working_days', 'weekend_days', 'booking_date', 'trip_end_dow', 'motivo_de_viaje', 'total_revenue', 'channel'] # , 'has_ins', 'has_pb', 'has_flex'

    df.drop(columns=[c for c in cols_drop if c in df.columns], inplace=True, errors='ignore')
    print(f"Columnas eliminadas: {len(cols_drop)}")


    return df

In [92]:
df = pd.read_csv('DataSet.csv', sep=';')

In [93]:
# B. Control de Daños (Tu función original)
control_de_danos(df)



Porcentaje de NaN por columna:
                            Total_values  NaN_values  Completeness  \
promocode                          30428       28325          2103   
has_weekends                       30428       14643         15785   
length_of_stay                     30428       14636         15792   
trip_end_dow                       30428       14636         15792   
working_days                       30428       14636         15792   
weekend_days                       30428       14636         15792   
bookingid                          30428           0         30428   
recordlocator                      30428           0         30428   
booking_dow                        30428           0         30428   
booking_hour                       30428           0         30428   
motivo_de_viaje                    30428           0         30428   
pos                                30428           0         30428   
rt_group                           30428           0      

In [94]:
# C. Aplicar Preparación Maestra
df_clean = preparacion_maestra(df)


Columnas eliminadas: 21


In [95]:
# D. Reducción de Memoria Final
control_de_danos(df_clean)
df_clean = reduce_mem_usage(df_clean)



Porcentaje de NaN por columna:
                         Total_values  NaN_values  Completeness  missing_pct
pos                             30428           0         30428          0.0
rt_group                        30428           0         30428          0.0
days_to_departure               30428           0         30428          0.0
rt_cc                           30428           0         30428          0.0
trip_rt                         30428           0         30428          0.0
trip_origin_city                30428           0         30428          0.0
trip_destination_city           30428           0         30428          0.0
trip_type                       30428           0         30428          0.0
length_of_stay                  30428           0         30428          0.0
trip_start_dow                  30428           0         30428          0.0
has_weekends                    30428           0         30428          0.0
paxs                            30428       

In [96]:
df_clean_general = df_clean.copy()

# Separar X e y ANTES de codificar
X_general = df_clean_general.drop('target', axis=1)
y_general = df_clean_general['target']

# Dividir en train/test
X_train_gen, X_test_gen, y_train_gen, y_test_gen = train_test_split(
    X_general, y_general, test_size=TEST_SIZE_GENERAL, random_state=SEED, stratify=y_general
)

preprocessor_general = ColumnTransformer(
    transformers=[
        ('target_encoder', ce.TargetEncoder(), COLS_TARGET_ENC),
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), COLS_ONE_HOT)
    ],
    remainder='passthrough', # Mantiene las columnas no especificadas (numéricas)
    verbose_feature_names_out=False
).set_output(transform="pandas")

# Ajustar el preprocesador SOLO con los datos de entrenamiento
preprocessor_general.fit(X_train_gen, y_train_gen)

# Transformar ambos conjuntos
X_train_gen_encoded = preprocessor_general.transform(X_train_gen)
X_test_gen_encoded = preprocessor_general.transform(X_test_gen)

json_path = f'{DATA_DIR}target_encoding_gen.json'
exportar_target_encoding_a_json(X_train_gen, y_train_gen, COLS_TARGET_ENC, json_path)

# Guardar archivos
X_train_gen_encoded.to_parquet(f'{DATA_DIR}X_train_general_encoded.parquet')
X_test_gen_encoded.to_parquet(f'{DATA_DIR}X_test_general_encoded.parquet')
y_train_gen.to_pickle(f'{DATA_DIR}y_train_general.pkl')
y_test_gen.to_pickle(f'{DATA_DIR}y_test_general.pkl')
joblib.dump(preprocessor_general, f'{DATA_DIR}preprocessor_general.pkl')

['Data/preprocessor_general.pkl']

In [97]:
# --- 2. PROCESAMIENTO PARA MODELOS POR PAÍS ---
print("\nProcesando y guardando para Modelos por País...")
paises = ['CL', 'AR', 'PE', 'CO'] # , 'INTER'

for pais in paises:
    df_pais = df_clean[df_clean['pos'] == pais].copy()
    if df_pais.empty:
        continue

    print(f"Procesando para: {pais}")

    # Columnas para este país (excluimos 'pos' ya que es constante)
    cols_target_pais = [c for c in COLS_TARGET_ENC if c != 'pos']
    cols_onehot_pais = COLS_ONE_HOT

    X_pais = df_pais.drop(['target', 'pos'], axis=1)
    y_pais = df_pais['target']

    X_train_pais, X_test_pais, y_train_pais, y_test_pais = train_test_split(
        X_pais, y_pais, test_size=TEST_SIZE_PAIS, random_state=SEED, stratify=y_pais
    )

    # Preprocesador específico para el país
    preprocessor_pais = ColumnTransformer(
        transformers=[
            ('target_encoder', ce.TargetEncoder(), cols_target_pais),
            ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cols_onehot_pais)
        ],
        remainder='passthrough',
        verbose_feature_names_out=False
    ).set_output(transform="pandas")

    # Ajustar y transformar
    preprocessor_pais.fit(X_train_pais, y_train_pais)
    X_train_pais_encoded = preprocessor_pais.transform(X_train_pais)
    X_test_pais_encoded = preprocessor_pais.transform(X_test_pais)

    json_path = f'DiccTargetEncoder/target_encoding_{pais}.json'

    exportar_target_encoding_a_json(X_train_pais, y_train_pais, cols_target_pais, json_path)


    # Guardar archivos
    X_train_pais_encoded.to_parquet(f'{DATA_DIR}X_train_{pais}_encoded.parquet')
    X_test_pais_encoded.to_parquet(f'{DATA_DIR}X_test_{pais}_encoded.parquet')
    y_train_pais.to_pickle(f'{DATA_DIR}y_train_{pais}.pkl')
    y_test_pais.to_pickle(f'{DATA_DIR}y_test_{pais}.pkl')
    joblib.dump(preprocessor_pais, f'{DATA_DIR}preprocessor_{pais}.pkl')



Procesando y guardando para Modelos por País...
Procesando para: CL
Procesando para: AR
Procesando para: PE
Procesando para: CO


In [98]:
df_clean.to_csv(f'{DATA_DIR}df_full.csv', index=False, sep=';')

In [99]:
df_clean.to_parquet(f'{DATA_DIR}df_full.parquet')